In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("paultimothymooney/chest-xray-pneumonia")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/chest-xray-pneumonia


In [ ]:
# Step 1: Install and configure Kaggle
!pip install kaggle
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Step 2: Download Chest X-Ray dataset
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

# Step 3: Extract the dataset
!unzip chest-xray-pneumonia.zip -d chest_xray_pneumonia


In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

In [ ]:
# Paths for dataset
train_path = './chest_xray_pneumonia/chest_xray/train'
val_path = './chest_xray_pneumonia/chest_xray/val'
test_path = './chest_xray_pneumonia/chest_xray/test'

transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((32,32)),
    #transforms.RandomHorizontalFlip(),       # Flip images randomly
    #transforms.RandomRotation(10),          # Rotate within ±10 degrees
    transforms.ToTensor(),
    #transforms.Normalize(mean=[0.5], std=[0.5])
])


# Load datasets
train_dataset = ImageFolder(root=train_path, transform=transform)
val_dataset = ImageFolder(root=val_path, transform=transform)
test_dataset = ImageFolder(root=test_path, transform=transform)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=2, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False)

print(f"Train samples: {len(train_dataset)}, Validation samples: {len(val_dataset)}, Test samples: {len(test_dataset)}")


Train samples: 5216, Validation samples: 16, Test samples: 624


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim.lr_scheduler import CosineAnnealingLR

# Define Autoencoder (Conv-based)
class ConvAutoencoder(nn.Module):
    def __init__(self):
        super().__init__()

        # Encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),
            nn.ReLU()
        )

        # Decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 1, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

    def get_latent_features(self, x):
        return self.encoder(x)

# Classifier (CNN on latent features)
def LatentFeatureCNN(input_channels=128, num_classes=1):
    model = nn.Sequential(
        nn.Conv2d(input_channels, 64, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2, 2),
        nn.Flatten(),
        nn.Linear(128 * 2 * 2, 256),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(256, num_classes)
    )
    return model

def train_autoencoder(autoencoder, train_loader, device, epochs=5):
    optimizer = optim.AdamW(autoencoder.parameters(), lr=0.001)
    criterion = nn.MSELoss()

    autoencoder.train()
    for epoch in range(epochs):
        total_loss = 0
        for images, _ in train_loader:
            images = images.to(device)

            outputs = autoencoder(images)
            loss = criterion(outputs, images)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f"Autoencoder Epoch {epoch+1}, Loss: {total_loss / len(train_loader):.4f}")






In [ ]:
def train_classifier(autoencoder, classifier, train_loader, val_loader, device, epochs=10):
    optimizer = optim.AdamW(classifier.parameters(), lr=0.001)
    scheduler = CosineAnnealingLR(optimizer, T_max=epochs)
    criterion = nn.BCEWithLogitsLoss()

    # Freeze Autoencoder Encoder
    for param in autoencoder.encoder.parameters():
        param.requires_grad = False

    for epoch in range(epochs):
        classifier.train()
        total_loss, correct, total = 0, 0, 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.float().to(device)

            # Extract latent features
            with torch.no_grad():
                latent_features = autoencoder.get_latent_features(images)

            # Dynamically reshape latent features
            batch_size = latent_features.size(0)
            latent_features = latent_features.view(batch_size, *latent_features.shape[1:])

            #print(f"Latent shape: {latent_features.shape}")  # Debug shape

            # Ensure classifier input matches the latent shape
            outputs = classifier(latent_features).squeeze(1)

            # Compute loss
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Metrics
            total_loss += loss.item() * images.size(0)
            preds = (torch.sigmoid(outputs) > 0.5).float()
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        scheduler.step()

        train_loss = total_loss / total
        train_acc = correct / total

        print(f"Epoch {epoch+1}/{epochs}, Loss: {train_loss:.4f}, Accuracy: {train_acc:.4f}")

        # Validation
        validate(classifier, autoencoder, val_loader, criterion, device)

def validate(classifier, autoencoder, val_loader, criterion, device):
    classifier.eval()
    total_loss, correct, total = 0, 0, 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.float().to(device)

            latent_features = autoencoder.get_latent_features(images)
            latent_features = latent_features.view(images.size(0), 128, 4, 4)

            outputs = classifier(latent_features).squeeze(1)

            loss = criterion(outputs, labels)

            total_loss += loss.item() * images.size(0)
            preds = (torch.sigmoid(outputs) > 0.5).float()
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    val_loss = total_loss / total
    val_acc = correct / total

    print(f"Validation Loss: {val_loss:.4f}, Accuracy: {val_acc:.4f}")

# Initialize models and device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
autoencoder = ConvAutoencoder().to(device)
classifier = LatentFeatureCNN().to(device)

# Train autoencoder
train_autoencoder(autoencoder, train_loader, device, epochs=3)

# Train classifier with autoencoder features
train_classifier(autoencoder, classifier, train_loader, val_loader, device, epochs=20)

Autoencoder Epoch 1, Loss: 0.0030
Autoencoder Epoch 2, Loss: 0.0010
Autoencoder Epoch 3, Loss: 0.0007
Epoch 1/20, Loss: 0.2289, Accuracy: 0.9089
Validation Loss: 0.5592, Accuracy: 0.8125
Epoch 2/20, Loss: 0.1287, Accuracy: 0.9507
Validation Loss: 0.7007, Accuracy: 0.7500
Epoch 3/20, Loss: 0.1100, Accuracy: 0.9605
Validation Loss: 0.2991, Accuracy: 0.8125
Epoch 4/20, Loss: 0.1055, Accuracy: 0.9605
Validation Loss: 0.1915, Accuracy: 0.8750
Epoch 5/20, Loss: 0.0913, Accuracy: 0.9663
Validation Loss: 0.3494, Accuracy: 0.8125
Epoch 6/20, Loss: 0.0888, Accuracy: 0.9641
Validation Loss: 0.1990, Accuracy: 0.9375
Epoch 7/20, Loss: 0.0784, Accuracy: 0.9724
Validation Loss: 0.4288, Accuracy: 0.7500
Epoch 8/20, Loss: 0.0767, Accuracy: 0.9720
Validation Loss: 1.9537, Accuracy: 0.6250
Epoch 9/20, Loss: 0.0748, Accuracy: 0.9734
Validation Loss: 0.6706, Accuracy: 0.6875
Epoch 10/20, Loss: 0.0669, Accuracy: 0.9747
Validation Loss: 0.3203, Accuracy: 0.8125
Epoch 11/20, Loss: 0.0591, Accuracy: 0.9781
Val

In [ ]:
# Testing Function
criterion = nn.BCEWithLogitsLoss()
def test_model(model,autoencoder,criterion,test_loader):
    model.eval()
    test_correct, test_total = 0, 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.float().to(device)

            latent_features = autoencoder.get_latent_features(images)
            latent_features = latent_features.view(images.size(0), 128, 4, 4)

            outputs = classifier(latent_features).squeeze(1)

            loss = criterion(outputs, labels)

            preds = (torch.sigmoid(outputs) > 0.5).float()
            test_correct += (preds == labels).sum().item()
            test_total += labels.size(0)

    test_acc = test_correct / test_total
    print(f"Test Accuracy: {test_acc:.4f}")

# Evaluate on test set
test_model(classifier,autoencoder,criterion,test_loader)

Test Accuracy: 0.7500
